<https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers>


In [1]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

In [2]:
import jax.numpy as jnp  # Oddly works in colab to set gpu

arr = jnp.array([1, 2, 3])
arr.devices()

{cuda(id=0)}

In [3]:
import icecream
from icecream import ic

icecream.install()
ic_disable = True
if ic_disable:
    ic.disable()

In [4]:
import os
from dataclasses import dataclass, field
from datetime import datetime as dt
from itertools import chain

import hephaestus as hp
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import optax
import pandas as pd
import seaborn as sns
from flax import linen as nn
from flax import struct  # Flax dataclasses
from flax.training import checkpoints, train_state
from flax.training.early_stopping import EarlyStopping
from icecream import ic
from jax import profiler, random
from jax.tree_util import tree_flatten
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange

pd.options.mode.copy_on_write = True

2024-03-11 02:24:59.103331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 02:24:59.103382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 02:24:59.104656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 02:24:59.846813: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
data = pd.read_parquet("data/processed_energy.parquet")

In [6]:
def sinify_datetime(df, col):
    df[col + "_base_year"] = df[col].dt.year
    df[col + "_day_sin"] = np.sin(2 * np.pi * df[col].dt.day / 365.25)
    df[col + "_day_cos"] = np.cos(2 * np.pi * df[col].dt.day / 365.25)
    df[col + "_hour_sin"] = np.sin(2 * np.pi * df[col].dt.hour / 24)
    df[col + "_hour_cos"] = np.cos(2 * np.pi * df[col].dt.hour / 24)
    df[col + "_is_weekend"] = (
        df[col].dt.dayofweek.isin([5, 6]).astype(str).map({"True": 1, "False": 0})
    )

    return df

In [7]:
batch_size = 3120 // 2 * 1
train_ds = hp.TabularTimeSeriesData(
    data.copy(),
    batch_size=batch_size,  # target_column="euros_per_mwh"
    target_column="target",
    train=True,
)

In [8]:
test_ds = hp.TabularTimeSeriesData(
    data.copy(),
    batch_size=batch_size,  # target_column="euros_per_mwh"
    target_column="target",
    train=False,
)

In [9]:
test_ds.n_cat_cols, train_ds.n_cat_cols

(4, 4)

In [10]:
time_series_regressor = hp.time_series.MaskedTimeSeriesRegClass(
    train_ds, d_model=64 * 4
)

In [11]:
def calculate_memory_footprint(params):
    """Calculate total memory footprint of JAX model parameters."""
    total_bytes = 0
    # Flatten the parameter tree structure into a list of arrays
    flat_params, _ = tree_flatten(params)
    for param in flat_params:
        # Calculate bytes: number of elements * size of each element
        bytes_per_param = param.size * param.dtype.itemsize
        total_bytes += bytes_per_param
    return total_bytes

In [12]:
@struct.dataclass
class Batch:
    numeric: jnp.ndarray
    categorical: jnp.ndarray
    y: jnp.ndarray

    @classmethod
    def from_ds_idx(
        cls,
        ds: hp.TabularTimeSeriesData,
        start: int = None,
        count: int = None,
        arr: list = None,
    ):
        numeric_list = []
        categorical_list = []
        y_list = []

        if (start is None and count is not None) or (
            start is not None and count is None
        ):
            if count is None:
                problem = "count"
            else:
                problem = "start"
            raise ValueError(
                "Both `count` and `start` must be defined or neither"
                + f"got only {problem} defined"
            )
        if (start is None) and (arr is None):
            raise ValueError("Do not provide start/count and arr values")

        if arr is not None:
            indexer = arr

        else:
            indexer = range(start + 1, start + count + 1)

        for idx in indexer:
            if arr is not None:
                idx = idx.item()
            categorical, numeric, y = ds[idx]
            numeric_list.append(numeric)
            categorical_list.append(categorical)
            y_list.append(y)

        numeric = jnp.array(numeric_list)
        categorical = jnp.array(categorical_list)
        y = jnp.array(y_list)
        return cls(numeric=numeric, categorical=categorical, y=y)


ts_train = Batch.from_ds_idx(train_ds, 0, 1)
ts_test = Batch.from_ds_idx(test_ds, 0, 1)
print(ts_train.numeric.shape, ts_test.numeric.shape)
print(ts_train.y.shape, ts_test.y.shape)
print(ts_train.categorical.shape, ts_test.categorical.shape)

assert ts_train.categorical.shape == ts_test.categorical.shape

(1, 1560, 65) (1, 1560, 65)
(1, 1560, 1) (1, 1560, 1)
(1, 1560, 4) (1, 1560, 4)


In [13]:
ts_train.categorical.shape, ts_test.categorical.shape

((1, 1560, 4), (1, 1560, 4))

In [14]:
# %%memit
# profiler.start_trace(
#     "/content/drive/MyDrive/Colab Notebooks/Hephaestus/runs"
# )
mts_root_key = random.PRNGKey(44)
mts_main_key, ts_params_key, ts_data_key = random.split(mts_root_key, 3)


def replace_nans(grads, replace_value=0):
    """Replace NaN values in gradients with a specified value."""
    return jax.tree_map(lambda g: jnp.where(jnp.isnan(g), replace_value, g), grads)


def clip_gradients(gradients, max_norm):
    total_norm = jnp.sqrt(sum(jnp.sum(jnp.square(grad)) for grad in gradients.values()))
    scale = max_norm / (total_norm + 1e-6)
    clipped_gradients = jax.tree_map(
        lambda grad: jnp.where(total_norm > max_norm, grad * scale, grad), gradients
    )
    return clipped_gradients


# @jax.jit
def calculate_loss(params, state, inputs, dataset, target: bool = True):
    numeric_loss_scale = 1.0
    target_loss_scale = 1.0
    ic("here")
    category_out, numeric_out, target_out = state.apply_fn(
        {"params": params},
        hp.mask_tensor(inputs.categorical, dataset, prng_key=ts_data_key),
        hp.mask_tensor(inputs.numeric, dataset, prng_key=ts_data_key),
    )
    ic("here")
    numeric_col_tokens = dataset.numeric_col_tokens.clone()
    numeric_col_tokens = numeric_col_tokens[None, None, :]
    repeated_numeric_col_tokens = jnp.tile(
        numeric_col_tokens, (category_out.shape[0], category_out.shape[1], 1)
    )

    ic(repeated_numeric_col_tokens.shape, inputs.categorical.shape, category_out.shape)
    categorical_targets = jnp.concatenate(
        [
            inputs.categorical,
            repeated_numeric_col_tokens,
        ],
        axis=-1,
    )
    ic(categorical_targets.shape)
    categorical_loss = optax.softmax_cross_entropy_with_integer_labels(
        category_out, categorical_targets
    ).mean()
    ic()
    target_loss = optax.squared_error(target_out, jnp.squeeze(inputs.y, -1)).mean()
    ic()
    numeric_loss = optax.squared_error(numeric_out, inputs.numeric).mean()
    ic()
    if target:
        total_loss = (
            categorical_loss
            + numeric_loss * numeric_loss_scale
            + target_loss * target_loss_scale
        )
    else:
        total_loss = categorical_loss + numeric_loss * numeric_loss_scale

    ic(total_loss, type(total_loss))
    return total_loss, {
        "target_loss": target_loss.astype(float),
        "numeric_loss": numeric_loss.astype(float),
        "categorical_loss": categorical_loss.astype(float),
    }


@jax.jit
def eval_step(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, loss_dict = loss_fn(state.params)
    return loss, loss_dict


@jax.jit
def eval_step_no_target(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds, target=False)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, loss_dict = loss_fn(state.params)
    return loss, loss_dict


@jax.jit
def train_step(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds)

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)

    # (loss, individual_losses), grad = grad_fn(state.params)
    (loss, loss_dict), grad = grad_fn(state.params)
    # grad = replace_nans(grad)
    # grad = clip_gradients(grad, 1.0)
    state = state.apply_gradients(grads=grad)

    return state, loss, loss_dict


@jax.jit
def train_step_no_target(state: train_state.TrainState, batch):
    def loss_fn(params):
        return calculate_loss(params, state, batch, train_ds, target=False)

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)

    # (loss, individual_losses), grad = grad_fn(state.params)
    (loss, loss_dict), grad = grad_fn(state.params)
    # grad = replace_nans(grad)
    # grad = clip_gradients(grad, 1.0)
    state = state.apply_gradients(grads=grad)

    return state, loss, loss_dict


def create_train_state(model, prng, batch, lr):
    params = model.init(prng, batch.categorical, batch.numeric)
    # optimizer = optax.chain(optax.adam(lr))
    optimizer = optax.chain(optax.clip_by_global_norm(0.4), optax.adam(lr))
    # optimizer_state = optimizer.init(params)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params["params"],
        tx=optimizer,
        # tx_state=optimizer_state,
    )


batch_size = 2
batch = Batch.from_ds_idx(train_ds, 0, batch_size)

state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)

In [15]:
from jax import numpy as jnp

In [16]:
x = jnp.ones((2, 3120, 69, 4, 16))
print(x.reshape((2, 3120, 69, 4 * 16)).shape)
print(x.reshape((x.shape[0], x.shape[1], x.shape[2], -1)).shape)

(2, 3120, 69, 64)
(2, 3120, 69, 64)


In [17]:
# nn.enable_named_call()
# # with jax.profiler.trace("./runs", create_perfetto_link=False):
# # train_ds = hp.TabularTimeSeriesData(
# #     data.copy(),
# #     batch_size=3120 // 2,  # target_column="euros_per_mwh"
# #     target_column="target",
# # )
# batch = Batch.from_ds_idx(train_ds, 0, batch_size)

# jax.profiler.start_trace("runs/")
# for i in range(10):
#     state, loss, loss_dict = train_step(state, batch)

# jax.profiler.stop_trace()

In [18]:
# try:
#     jax.profiler.stop_trace()
# except RuntimeError as e:
#     print(e)

In [19]:
def show_mask_pred(
    state, batch: Batch, probability: int = 0.8, prng_key=ts_data_key, dataset=train_ds
):
    prng_key, subkey = random.split(prng_key)
    categorical_inputs = hp.mask_tensor(
        batch.categorical, dataset, prng_key=subkey, probability=probability
    )
    prng_key, subkey = random.split(prng_key)
    numeric_inputs = hp.mask_tensor(
        batch.numeric, dataset, prng_key=subkey, probability=probability
    )
    category_out, numeric_out, target_out = state.apply_fn(
        {"params": state.params}, categorical_inputs, numeric_inputs
    )  # TODO Make sure this is state.params and not something else.

    categorical_preds = category_out.argmax(axis=-1)

    # Get the words from the tokens
    decoder_dict = dataset.token_decoder_dict
    categorical_preds = [decoder_dict[i.item()] for i in categorical_preds[0]]

    results_dict = {k: cat_preds[i] for i, k in enumerate(dataset.col_tokens)}
    for i, k in enumerate(dataset.col_tokens[dataset.n_cat_cols :]):
        results_dict[k] = numeric_preds[0][i].item()
    # Get the masked values
    categorical_masked = [decoder_dict[i.item()] for i in categorical_inputs[0]]
    numeric_masked = numeric_inputs[0].tolist()
    masked_values = categorical_masked + numeric_masked
    # zip the masked values with the column names
    masked_dict = dict(zip(dataset.col_tokens, masked_values))
    # Get the original values
    categorical_values = [
        decoder_dict[i.item()]
        for i in mi.categorical_targets[0][0 : dataset.n_cat_cols]
    ]

    numeric_values = mi.numeric_targets.tolist()[0]
    original_values = categorical_values
    original_values.extend(numeric_values)
    # zip the original values with the column names
    original_dict = dict(zip(dataset.col_tokens, original_values))

    result_dict = {
        "masked": masked_dict,
        "actual": original_dict,
        "pred": results_dict,
    }

    return result_dict

In [20]:
# show_mask_pred(state, batch)

In [21]:
writer_name = "SkipFirst"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")

train_summary_writer = SummaryWriter(
    "runs/" + writer_time + "_" + writer_name + "_train"
)
test_summary_writer = SummaryWriter("runs/" + writer_time + "_" + writer_name + "_test")


def create_iterations(ds, batch_size, prng_key):
    arr = jnp.arange(len(ds))

    # Shuffle the array
    shuffled_arr = jax.random.permutation(prng_key, arr)
    num_groups = (
        arr.size + (batch_size - 1)
    ) // batch_size  # This ensures that the last group has <= batch_size elements

    # Split the shuffled array into groups
    groups = jnp.split(
        shuffled_arr, jnp.arange(batch_size, batch_size * num_groups, batch_size)
    )
    return groups


test_batch = Batch.from_ds_idx(test_ds, 0, 2)

test_set_key = random.PRNGKey(4454)


no_target_cutoff = 6
batch_count = 0
for j in trange(14, desc="epochs"):
    arrs = create_iterations(train_ds, 2, ts_data_key)
    test_arrs = create_iterations(test_ds, 2, ts_data_key)
    if j < no_target_cutoff:
        has_target = False
    else:
        has_target = True
    for i in tqdm(arrs, leave=False, desc=f"batches {has_target}"):
        # for i in trange(len(pre_train) // batch_size, leave=False):
        # for i in trange(len(pre_train) // batch_size //10, leave=False):
        batch = Batch.from_ds_idx(ds=train_ds, arr=i)
        if has_target:
            state, loss, loss_dict = train_step(state, batch)
        else:
            state, loss, loss_dict = train_step_no_target(state, batch)
        batch_count += 1

        if batch_count % 10 == 0:
            train_summary_writer.add_scalar(
                "loss/loss", np.array(loss.item()), batch_count
            )
            train_summary_writer.add_scalar(
                "loss/categorical", np.array(loss_dict["categorical_loss"]), batch_count
            )
            train_summary_writer.add_scalar(
                "loss/numeric", np.array(loss_dict["numeric_loss"]), batch_count
            )
            train_summary_writer.add_scalar(
                "loss/target", np.array(loss_dict["target_loss"]), batch_count
            )
            test_set_key, sub_test_key = random.split(test_set_key)
            random_index = random.randint(
                sub_test_key, (1,), 0, len(test_ds) - 2
            ).item()
            test_batch = Batch.from_ds_idx(test_ds, random_index, 2)
            ## Testing
            if has_target:
                test_loss, test_loss_dict = eval_step(state, test_batch)
            else:
                test_loss, test_loss_dict = eval_step_no_target(state, test_batch)
            test_summary_writer.add_scalar(
                "loss/has_target", np.array(float(has_target)), batch_count
            )
            test_summary_writer.add_scalar(
                "loss/loss", np.array(test_loss.item()), batch_count
            )
            test_summary_writer.add_scalar(
                "loss/categorical",
                np.array(test_loss_dict["categorical_loss"]),
                batch_count,
            )
            test_summary_writer.add_scalar(
                "loss/numeric", np.array(test_loss_dict["numeric_loss"]), batch_count
            )
            test_summary_writer.add_scalar(
                "loss/target", np.array(test_loss_dict["target_loss"]), batch_count
            )


# profiler.stop_trace()

epochs:   0%|          | 0/14 [00:00<?, ?it/s]

batches False:   0%|          | 0/517 [00:00<?, ?it/s]

batches False:   0%|          | 0/517 [00:00<?, ?it/s]

batches False:   0%|          | 0/517 [00:00<?, ?it/s]

batches False:   0%|          | 0/517 [00:00<?, ?it/s]

...

batches True:   0%|          | 0/517 [00:00<?, ?it/s]

batches True:   0%|          | 0/517 [00:00<?, ?it/s]

batches True:   0%|          | 0/517 [00:00<?, ?it/s]

batches True:   0%|          | 0/517 [00:00<?, ?it/s]

batches True:   0%|          | 0/517 [00:00<?, ?it/s]

In [22]:
ts_data_key = random.PRNGKey(1)
ts_data_key, new_key = random.split(ts_data_key)

In [23]:
for i in range(4):
    new_key, subkey = random.split(new_key)
    print(subkey, new_key)
    random_index = random.randint(new_key, (1,), 0, len(test_ds) - 1).item()
    print(random_index)

[2554108775 1009840025] [2986633504  617412143]
145
[3295656527 1572682181] [ 809688650 2172309036]
138
[ 612812304 3708077227] [3547667909 2943765984]
177
[703307542 615583408] [3106088119 3916689784]
197


In [24]:
model_name = f"big_train_{dt.now()}_"

current_dir = os.getcwd()

if not os.path.exists(f"./pre_trained_models/"):
    os.makedirs(f"./pre_trained_models/")

path = os.path.join(current_dir, f"./pre_trained_models/")


ckpt_dir = f"./pre_trained_models/{model_name}"

# checkpoints.save_checkpoint(
#     ckpt_dir=path, target=state, step=batch_count, overwrite=True, prefix=model_name
# )

In [25]:
def jax_array_memory_usage(array):
    """Calculate the memory usage of a JAX array in bytes."""
    # Get the number of bytes per element based on the data type
    bytes_per_element = array.dtype.itemsize
    # Calculate the total number of elements in the array
    total_elements = np.prod(array.shape)
    # Calculate total memory usage
    memory_usage_bytes = bytes_per_element * total_elements
    return memory_usage_bytes


cat_memory_usage = jax_array_memory_usage(batch.categorical)
num_memory_usage = jax_array_memory_usage(batch.numeric)
memory_usage = cat_memory_usage + num_memory_usage
memory_usage_gb = memory_usage / 1024 / 1024 / 1024
print(f"Memory usage: {memory_usage} bytes")
print(f"Memory usage: {memory_usage_gb} gb")

Memory usage: 861120 bytes
Memory usage: 0.0008019804954528809 gb
